In [1]:
import re
import requests
from bs4 import BeautifulSoup
from OSGridConverter import grid2latlong


In [2]:
import os
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
import django
django.setup()

In [3]:
from castles.models import Castle

In [21]:
from django.db import connection
connection.connect()

In [8]:
cas = Castle.objects.all()
cas[0].canmore_map

In [ ]:
url='https://thecastleguy.co.uk/castle/?cpage='
page=1

In [ ]:
html = requests.get(f"{url}{page}")

In [ ]:
soup = BeautifulSoup(html.text, 'html.parser')

In [ ]:
cas=soup.find_all('div', class_='castle')
cas

In [ ]:

n=1
for page in range(1,37):
    html = requests.get(f"{url}{page}")
    soup = BeautifulSoup(html.text, 'html.parser')
    cas=soup.find_all('div', class_='castle')
    for c in cas:
        name = c.find('p',class_='castle-name').text.strip()
        source_url = c.find('p',class_='castle-name').find('a').get('href')
        print (n,'*Name:',name,'*Source_url:',source_url)
        castle = Castle(source='thecastleguy.co.uk',source_url=source_url,name=name)
        for info in c.findAll('li'):
            #print (info.text.strip())

            inforeg = re.search(r'(.*)\s*:\s*(.*)',info.text.strip())
            if inforeg:
                #print (inforeg.group(1),inforeg.group(2))
                if inforeg.group(1)=='Grid Reference':
                    castle.grid_reference = inforeg.group(2)
                    l=grid2latlong(castle.grid_reference)
                    castle.lat = l.latitude
                    castle.lng = l.longitude
                    print ('grid_reference',inforeg.group(2),'lat:',castle.lat,'lng:',castle.lng)
                elif inforeg.group(1)=='Closest To':
                    print ('closest_to',inforeg.group(2))
                    castle.closest_to = inforeg.group(2)
            else:
                print ('access',info.text.strip())
                castle.access = info.text.strip()


        print(c.find('p',class_='castle-index').text.strip())
        castle.description_short = c.find('p',class_='castle-index').text.strip()
        print (c.find('img').get('src'))
        castle.image_url = c.find('img').get('src')

        castle.save()

        n+=1

In [ ]:
from OSGridConverter import grid2latlong

In [ ]:
l=grid2latlong('NO301410')
(l.latitude,l.longitude)

In [ ]:
cc = Castle.objects.all()
for c in cc:
    l=grid2latlong(c.grid_reference)
    print (c.name,c.grid_reference,l.latitude,l.longitude)
    c.lat = l.latitude
    c.lng = l.longitude
    c.save()


In [22]:
cc = Castle.objects.filter(canmore_url__isnull=True)
for c in cc:
    print (c.name,c.source_url)
    html = requests.get(c.source_url)
    soup = BeautifulSoup(html.text, 'html.parser')
    #soup.find_all('a', title="HES Canmore database entry")
    cas=soup.select_one("a[href*='canmore.org.uk/site/']")
    if cas : 
        print (cas['href'])
        if cas['href'].startswith('http'):
            c.canmore_url = cas['href']
        else:
            c.canmore_url = 'https://'+cas['href']
        html = requests.get(c.canmore_url)
        soup = BeautifulSoup(html.text, 'html.parser')
        grid = soup.find(text="NGR")
        ref=grid.parent.parent.text.replace('NGR','').strip()
        c.grid_reference = ref
        #datum = soup.find(text="Datum")
        #datum=datum.parent.parent.text.replace('Datum','').replace('NGR','').replace('-','').strip()
        #l=grid2latlong(ref,tag=datum) 
        l=grid2latlong(ref) 
        c.lat = l.latitude
        c.lng = l.longitude
        map=soup.select_one("a[href*='view=map']")
        print(ref,c.grid_reference,l,map)
        c.canmore_map = 'https://canmore.org.uk' + map['href']
        

        c.save()

    #c.save()



Auchen Castle https://thecastleguy.co.uk/castle/auchen-castle/
Berwick Castle https://thecastleguy.co.uk/castle/berwick-castle/
Blairfindy Castle https://thecastleguy.co.uk/castle/blairfindy-castle/
Lochwood Castle https://thecastleguy.co.uk/castle/lochwood-castle/
Raitcastle https://thecastleguy.co.uk/castle/raitcastle/


In [11]:
cc[0].canmore_map

'/site/search/result?NUMLINK=65463&view=map'